Ce notebook a pour objectif de tester quelques LLM:

1- **Llama-2-7B-AWQ:**	Llama 2(ormé entre janvier 2023 et juillet 2023.) est une collection de modèles de texte génératifs pré-entraînés et affinés, dont l'échelle varie de 7 à 70 milliards de paramètres. Il s'agit du référentiel pour le modèle pré-entraîné 7B, converti pour le format Hugging Face Transformers.  
Le modèle prend en entrée du texte et retourne uniquement du texte.  
Llama 2 est un modèle linguistique auto-régressif qui utilise une architecture de transformateur optimisée. Les versions optimisées utilisent le supervised fine-tuning (SFT) et l'apprentissage par renforcement avec retour d'information humain (RLHF) pour s'aligner sur les préférences humaines en matière de serviabilité et de sécurité.

Noter que AWQ est une méthode efficace, précise et extrêmement rapide de quantification par poids à faible bit, qui prend actuellement en charge la quantification à 4 bits. Par rapport à GPTQ, elle offre une inférence plus rapide basée sur les transformateurs. L'utilisation d'AWQ permet d'utiliser des GPU beaucoup plus petits, ce qui peut faciliter le déploiement et réduire les coûts globaux. Par exemple, un modèle 70B peut être exécuté sur 1 GPU de 48 Go au lieu de 2 GPU de 80 Go.

2- **Mistral:** Le Mistral-7B-v0.1 Large Language Model (LLM) est un modèle de texte génératif pré-entraîné avec 7 milliards de paramètres. Mistral-7B-v0.1 surpasse Llama 2 13B sur tous les points de référence testés(d'apres la page officielle: connaissance, raisonnement, compréhension, mathématiques,…). Mixtral est multilingue (anglais, français, italien, allemand, espagnol), se montrerait performant en programmation informatique et pourrait gérer une fenêtre de contexte (quantité de texte traiter en une fois) de 32’000 tokens (environ 25’000 mots). Mixtral a la particularité de posséder 46,7 milliards de paramètres au total, mais en n'utilisant que 12,9 milliards de paramètres par token. Car le modèle repose sur une approche dite «Mixture of Experts» (MOE) qui permet d'augmenter le nombre de paramètres d'un modèle tout en contrôlant le coût et la latence.

l’architecture MOE est composée de réseaux neuronaux distincts, nommés «experts», qui traitent chacun un sous-ensemble différent de données d'apprentissage. Une autre couche (un réseau de passerelles ou routeur) supervise et orchestre le distribution des tokens à ces différents experts, selon le poids de leur expertise pour une tâche donnée. A noter qu’il est possible de transmettre un token à plus d'un expert. Au cours de la formation du modèle, à la fois les experts et le système de délégation des tâches sont entraînés.

Selon les chercheurs de Mistral, les MOE présentent l’avantage de permettre un pré-entraînement nettement plus efficace en termes de ressources de calcul. En outre, l'inférence est beaucoup plus rapide qu'avec d’autres modèles comportant le même nombre de paramètres.

lien drive: https://drive.google.com/drive/folders/1gA4GrkLDmzHAUa7lUIFI2al4NZcLF1WP?usp=sharing

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd '/content/drive/MyDrive/M2Datascale/Projet_Transversal/TestLLM'

/content/drive/MyDrive/M2Datascale/Projet_Transversal/TestLLM


In [ ]:
%pip install transformers autoawq ctransformers

In [28]:
import time
import pandas as pd

In [29]:
questions = pd.read_excel('questionsllm.xlsx')
data = questions.sample(n=5)

In [30]:
data

,Question,Answers
40,How is COPD diagnosed?,Ongoing studies on antibiotic resistance.
34,How can I manage my Asthma?,"No specific cure, supportive care."
257,What are the risk factors for Kidney Stones?,Treatment options for Endometriosis include me...
147,Can Stroke be prevented?,Treatment options for Stroke include clot-bust...
281,What causes Glaucoma?,Glaucoma is often caused by increased intraocu...


### llama-index

In [9]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("TheBloke/Llama-2-7B-AWQ")
model = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7B-AWQ",trust_remote_code=True, device_map='auto')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [33]:
def getLlamaResponse(question,reponse_attendue):
  prompt_template=f'''{question}

  '''
  tokens = tokenizer(
      prompt_template,
      return_tensors='pt'
  ).input_ids.cuda()

  # Generate output
  start_time=time.time()
  generation_output = model.generate(
      tokens,
      do_sample=True,
      temperature=0.7,
      top_p=0.95,
      top_k=40,
      max_new_tokens=512
  )
  response = tokenizer.decode(generation_output[0])
  duree = time.time() - start_time
  new_row={'Model':'Llama-2-7B-AWQ', 'Questions':question, 'Reponses':response, 'Temps':duree, 'Reponses_Attendues':reponse_attendue}
  return new_row

In [ ]:
list_data_llama=[]
for i in range(len(data)):
   list_data_llama.append(getLlamaResponse(data.iloc[i]['Question'],data.iloc[i]['Answers']))
list_data_llama

In [35]:
df_llama = pd.DataFrame(data=list_data_llama)

In [36]:
df_llama

,Model,Questions,Reponses,Temps,Reponses_Attendues
0,Llama-2-7B-AWQ,How is COPD diagnosed?,<s> How is COPD diagnosed?\n\n \n 1. You're...,27.745727,Ongoing studies on antibiotic resistance.
1,Llama-2-7B-AWQ,How can I manage my Asthma?,<s> How can I manage my Asthma?\n\n \n\n\n\n\...,23.063081,"No specific cure, supportive care."
2,Llama-2-7B-AWQ,What are the risk factors for Kidney Stones?,<s> What are the risk factors for Kidney Stone...,23.538991,Treatment options for Endometriosis include me...
3,Llama-2-7B-AWQ,Can Stroke be prevented?,<s> Can Stroke be prevented?\n\n Yes. A stro...,23.082371,Treatment options for Stroke include clot-bust...
4,Llama-2-7B-AWQ,What causes Glaucoma?,<s> What causes Glaucoma?\n\n \n\nGlaucoma is...,23.196999,Glaucoma is often caused by increased intraocu...


# Mistral

In [37]:
from ctransformers import AutoModelForCausalLM
# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-v0.1-GGUF", model_file="mistral-7b-v0.1.Q4_K_M.gguf", model_type="mistral", gpu_layers=50)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

mistral-7b-v0.1.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

In [40]:
def getMistralResponse(question,reponse_attendue):
  start_time = time.time()
  response = llm(question)
  duree = (time.time() - start_time)
  new_row={'Model':'Mistral-7B', 'Questions':question, 'Reponses':response, 'Temps':duree, 'Reponses_Attendues':reponse_attendue}
  return new_row

In [41]:
list_data_mistral=[]
for i in range(len(data)):
   list_data_mistral.append(getMistralResponse(data.iloc[i]['Question'],data.iloc[i]['Answers']))

In [42]:
df_mistral = pd.DataFrame(data=list_data_mistral)

In [43]:
df_mistral

,Model,Questions,Reponses,Temps,Reponses_Attendues
0,Mistral-7B,How is COPD diagnosed?,\n\nCOPD is diagnosed by a doctor based on you...,16.782794,Ongoing studies on antibiotic resistance.
1,Mistral-7B,How can I manage my Asthma?,\n\n- Keep a written asthma action plan\n -...,9.917834,"No specific cure, supportive care."
2,Mistral-7B,What are the risk factors for Kidney Stones?,\n\n1. The following can be some of the major ...,13.712001,Treatment options for Endometriosis include me...
3,Mistral-7B,Can Stroke be prevented?,What should be the prevention for it and why ...,13.023208,Treatment options for Stroke include clot-bust...
4,Mistral-7B,What causes Glaucoma?,\n\nGlaucoma is caused by damage to the optic ...,12.688796,Glaucoma is often caused by increased intraocu...


In [47]:
df = df_mistral.append(df_llama, ignore_index=True)

<ipython-input-47-6d02ffdcd12f>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_mistral.append(df_llama, ignore_index=True)


In [48]:
df

,Model,Questions,Reponses,Temps,Reponses_Attendues
0,Mistral-7B,How is COPD diagnosed?,\n\nCOPD is diagnosed by a doctor based on you...,16.782794,Ongoing studies on antibiotic resistance.
1,Mistral-7B,How can I manage my Asthma?,\n\n- Keep a written asthma action plan\n -...,9.917834,"No specific cure, supportive care."
2,Mistral-7B,What are the risk factors for Kidney Stones?,\n\n1. The following can be some of the major ...,13.712001,Treatment options for Endometriosis include me...
3,Mistral-7B,Can Stroke be prevented?,What should be the prevention for it and why ...,13.023208,Treatment options for Stroke include clot-bust...
4,Mistral-7B,What causes Glaucoma?,\n\nGlaucoma is caused by damage to the optic ...,12.688796,Glaucoma is often caused by increased intraocu...
5,Llama-2-7B-AWQ,How is COPD diagnosed?,<s> How is COPD diagnosed?\n\n \n 1. You're...,27.745727,Ongoing studies on antibiotic resistance.
6,Llama-2-7B-AWQ,How can I manage my Asthma?,<s> How can I manage my Asthma?\n\n \n\n\n\n\...,23.063081,"No specific cure, supportive care."
7,Llama-2-7B-AWQ,What are the risk factors for Kidney Stones?,<s> What are the risk factors for Kidney Stone...,23.538991,Treatment options for Endometriosis include me...
8,Llama-2-7B-AWQ,Can Stroke be prevented?,<s> Can Stroke be prevented?\n\n Yes. A stro...,23.082371,Treatment options for Stroke include clot-bust...
9,Llama-2-7B-AWQ,What causes Glaucoma?,<s> What causes Glaucoma?\n\n \n\nGlaucoma is...,23.196999,Glaucoma is often caused by increased intraocu...


In [49]:
df.to_csv('llmResults.csv')